# Metrics for cubes

This notebooks computes and shows various metrics for each cube.

Pseudocode of this notebook looks like:

```python
for each cube:
    mkdir
    infer geometry
    if SHOW, log to std.out
    
    for each metric:
        compute metric
        save png
        if SAVE_POINT_CLOUDS, save point cloud
        if SHOW, draw in std.out  
```

#### There are following parameters: 
* `paths` controls which cubes are used
*  `metrics` allows to choose computed metrics
* `script_dir` is a location to save metric maps
* `local_kwargs`, `support_kwargs` define exact parameters of metric computation
* `ADD_PREFIX` stores parameters of metric computation in the name of saved metric map
* `SAVE_POINT_CLOUD` stores obtained values next to the metric map
* `SHOW` controls whether images are displayed in the notebook

In [ ]:
import os
import sys
from glob import glob
from tqdm.auto import tqdm
from copy import copy

sys.path.append('..')
from seismiqb import SeismicGeometry, GeometryMetrics, plot_image, save_point_cloud

In [ ]:
# Cubes to evaluate
paths = sorted(glob('/data/seismic_data/seismic_interpretation/CUBE_*/amp*.hdf5'))
[print(path) for path in paths]

# Metrics to compute
# metrics = GeometryMetrics.AVAILABLE_METRICS
metrics = ['support_hellinger', 'support_corrs',]

In [ ]:
# Directory for all the files to be saved in
# For each cube, a separate dir is created inside
script_dir = 'cube_metrics'
try: os.mkdir(script_dir)
except FileExistsError: pass


# Parameters for `local` metrics like `local_correlation`
local_kwargs = {
    'agg': None,
    'kernel_size': 3,
    'reduce_func': 'mean',
}

# Parameters for `support` metrics like `support_hellinger`
support_kwargs = {
    'agg': 'mean',
    'supports': 20,
}

# Parameters of drawing
plot_kwargs = {
    'cmap': 'Reds',
    'figsize': (15, 15),
}


# Whether file names contain info about parameters of metric computation
ADD_PREFIX = True

# Whether to save point clouds of metrics right next to the images
SAVE_POINT_CLOUDS = False

# Whether to log progress/draw images to std.out
SHOW = True

In [ ]:
%%time
for path_cube in tqdm(paths, ncols=1100):
    
    geometry = SeismicGeometry(path_cube)
    gm = GeometryMetrics(geometry)
    
    cube_dir = '/'.join((script_dir, geometry.short_name))
    try: os.mkdir(cube_dir)
    except FileExistsError: pass
    
    if SHOW:
        print('▆'*130); print('▆'*130);
        print(geometry)
    
    computed_metrics = []
    for metric_name in tqdm(metrics, ncols=600):        
        save_path = '/'.join((cube_dir, metric_name))
        kwargs = copy(local_kwargs) if metric_name.startswith('local') else copy(support_kwargs)

        if ADD_PREFIX:
            save_path += '@'
            save_path += '|'.join([':'.join([key,str(value)])
                                   for key, value in kwargs.items()])
        
        metric = gm.evaluate(metric_name, **kwargs,
                             plot=True, show_plot=SHOW, plot_kwargs=plot_kwargs,
                             savepath=save_path + '.png')
        computed_metrics.append(metric)
        
        if SAVE_POINT_CLOUDS:
            save_point_cloud(metric, save_path, geometry)

            
    save_path = '/'.join((cube_dir, 'quality_map'))
    if ADD_PREFIX:
        save_path += '@'
        save_path += '|'.join([':'.join([key,str(value)])
                               for key, value in kwargs.items()])
    quality_map = gm.evaluate('quality_map', quantiles=[0.1], agg=None,
                              computed_metrics=computed_metrics,
                              plot=True, show_plot=SHOW, plot_kwargs=plot_kwargs,
                              savepath=save_path + '.png')
    plt.show()